# Домашнее задание №3

## Урок 3. Коллаборативная фильтрация

In [1]:
# !pip install implicit
# !pip install src

 Downloading implicit-0.6.1-cp39-cp39-manylinux2014_x86_64.whl (18.6 MB)

In [2]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Метрики
# from src.metrics import precision_at_k, recall_at_k
# from implicit.evaluation import precision_at_k

import warnings
warnings.filterwarnings('ignore')

/home/gans/anaconda3/lib/python3.9/site-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: forward compatibility was attempted on non supported HW (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [3]:
data = pd.read_csv('./data/retail_train.csv')

test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    precision = flags.sum() / len(recommended_list)
        
    return precision


def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list 
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)    
    recall = flags.sum() / len(bought_list)
    
    return recall

In [5]:
item_features = pd.read_csv('./data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [6]:
print(f"Количество отделов: \n {item_features['department'].nunique()}")
print(f'Перечень отделов: \n {item_features.department.unique()}')

Количество отделов: 
 44
Перечень отделов: 
 ['GROCERY' 'MISC. TRANS.' 'PASTRY' 'DRUG GM' 'MEAT-PCKGD' 'SEAFOOD-PCKGD'
 'PRODUCE' 'NUTRITION' 'DELI' 'COSMETICS' 'MEAT' 'FLORAL'
 'TRAVEL & LEISUR' 'SEAFOOD' 'MISC SALES TRAN' 'SALAD BAR' 'KIOSK-GAS'
 'ELECT &PLUMBING' 'GRO BAKERY' 'GM MERCH EXP' 'FROZEN GROCERY'
 'COUP/STR & MFG' 'SPIRITS' 'GARDEN CENTER' 'TOYS' 'CHARITABLE CONT'
 'RESTAURANT' 'RX' 'PROD-WHS SALES' 'MEAT-WHSE' 'DAIRY DELI' 'CHEF SHOPPE'
 'HBC' 'DELI/SNACK BAR' 'PORK' 'AUTOMOTIVE' 'VIDEO RENTAL' ' '
 'CNTRL/STORE SUP' 'HOUSEWARES' 'POSTAL CENTER' 'PHOTO' 'VIDEO'
 'PHARMACY SUPPLY']


In [7]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns = ['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [8]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [9]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)

data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train,
                                 index='user_id',
                                 columns='item_id',
                                 values='quantity', # Можно пробовать другие варианты
                                 aggfunc='count',
                                 fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix (разреженной матрицы)

sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# заведём словари между соответствующими идентификаторами user item 
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

## ALS

In [11]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                # alpha=0.5,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4,                                
                                random_state=42)

model.fit(sparse_user_item,  # На вход item-user matrix
          show_progress=True)

  0%|          | 0/15 [00:00<?, ?it/s]

CPU times: user 5.09 s, sys: 18.9 ms, total: 5.11 s
Wall time: 1.36 s


In [12]:
recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=sparse_user_item,   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=[itemid_to_id[999999]], 
                        recalculate_user=False)

In [13]:
[id_to_itemid[rec] for rec in recs[0]]

[1106523, 1133018, 5569230, 1082185, 1068719]

In [14]:
def get_recommendations(user, model, sparse_user_item, N=5):
    res = [id_to_itemid[rec] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item[userid_to_id[user]],   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=False)[0]]
    return res

In [15]:
%%time
    
result['als'] = result['user_id'].map(lambda x: get_recommendations(x, model, sparse_user_item, N=5))
result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

CPU times: user 1.42 s, sys: 43.5 ms, total: 1.46 s
Wall time: 366 ms


0.1814887365328086

In [16]:
result.head(2)

,user_id,actual,als
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1105488, 5569374, 1033142, 979707, 995242]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 1133018, 910032, 5569327, 1022003]"


In [17]:
parameters = {'factors': [100, 200, 300],
              'regularization': [0.0005, 0.001, 0.01],
              'iterations': [10, 15, 30],
              'calculate_training_loss': [True],
              'num_threads': [4],
              'random_state': [42]}

In [18]:
def grid_search(X, y, class_model, parameters):
    
    best_params = {}
    best_score = 0
    
    for params in itertools.product(*parameters.values()):
        model = class_model(**dict(zip(parameters.keys(), params)))
        model.fit(X, show_progress=False)
        y['res'] = y['user_id'].apply(lambda x: get_recommendations(x, model=model, sparse_user_item=sparse_user_item, N=5))
        score = y.apply(lambda row: precision_at_k(row['res'], row['actual']), axis=1).mean()
        if score > best_score:
            best_score = score
            best_params = dict(zip(parameters.keys(), params))
        
    print(best_score)             
    return best_params    

### ALS без взвещивания

In [19]:
%%time

als_params = grid_search(sparse_user_item, 
                       result[['user_id', 'actual']], AlternatingLeastSquares, parameters)

0.18256611165523748
CPU times: user 4min 28s, sys: 674 ms, total: 4min 29s
Wall time: 1min 7s


In [20]:
als_params

{'factors': 100,
 'regularization': 0.01,
 'iterations': 10,
 'calculate_training_loss': True,
 'num_threads': 4,
 'random_state': 42}

In [32]:
%%time

model = AlternatingLeastSquares(**als_params)

model.fit(sparse_user_item,  # На вход item-user matrix
          show_progress=True)

result['als'] = result['user_id'].map(lambda x: get_recommendations(x, model, sparse_user_item, N=5))
result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()
result.head(3)

  0%|          | 0/10 [00:00<?, ?it/s]

CPU times: user 4.75 s, sys: 72 ms, total: 4.83 s
Wall time: 1.24 s


,user_id,actual,als
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5569374, 1033142, 979707, 1105488, 995242]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 1133018, 910032, 1022003, 938700]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1007195, 866211, 878996, 820165, 986912]"


### TF-IDF взвешивание

In [22]:
tfidf_user_item_matrix = tfidf_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [23]:
%%time

als_tfidf_params = grid_search(csr_matrix(tfidf_user_item_matrix), 
                             result[['user_id', 'actual']], AlternatingLeastSquares, parameters)

0.14407443682663865
CPU times: user 4min 36s, sys: 792 ms, total: 4min 37s
Wall time: 1min 9s


In [24]:
als_tfidf_params

{'factors': 100,
 'regularization': 0.01,
 'iterations': 10,
 'calculate_training_loss': True,
 'num_threads': 4,
 'random_state': 42}

In [33]:
%%time

model = AlternatingLeastSquares(**als_tfidf_params)

model.fit(tfidf_user_item_matrix,  # На вход item-user matrix
          show_progress=True)

result['als_tfidf'] = result['user_id'].map(lambda x: get_recommendations(x, model, sparse_user_item, N=5))
result.apply(lambda row: precision_at_k(row['als_tfidf'], row['actual']), axis=1).mean()
result.head(3)

  0%|          | 0/10 [00:00<?, ?it/s]

CPU times: user 4.79 s, sys: 24 ms, total: 4.81 s
Wall time: 1.23 s


,user_id,actual,als,als_tfidf
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5569374, 1033142, 979707, 1105488, 995242]","[1056509, 1041796, 844818, 1033142, 995242]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 1133018, 910032, 1022003, 938700]","[938700, 1124729, 1022003, 951590, 1051069]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1007195, 866211, 878996, 820165, 986912]","[1023720, 1007195, 1127179, 878996, 911878]"


#### BM25 взвешивание

In [26]:
bm25_user_item_matrix = bm25_weight(user_item_matrix.T).T.tocsr()  # Применяется к item-user матрице ! 

In [27]:
%%time

als_bm25_params = grid_search(csr_matrix(bm25_user_item_matrix).tocsr(), 
                               result[['user_id', 'actual']], AlternatingLeastSquares, parameters)

0.12272282076395585
CPU times: user 4min 34s, sys: 736 ms, total: 4min 34s
Wall time: 1min 8s


In [28]:
als_bm25_params

{'factors': 200,
 'regularization': 0.01,
 'iterations': 30,
 'calculate_training_loss': True,
 'num_threads': 4,
 'random_state': 42}

In [34]:
%%time

model = AlternatingLeastSquares(**als_bm25_params)

model.fit(csr_matrix(bm25_user_item_matrix),  # На вход item-user matrix
          show_progress=True)

result['als_bm25'] = result['user_id'].map(lambda x: get_recommendations(x, model, sparse_user_item, N=5))

result.apply(lambda row: precision_at_k(row['als_bm25'], row['actual']), axis=1).mean()
result.head(3)

  0%|          | 0/30 [00:00<?, ?it/s]

CPU times: user 16.7 s, sys: 24 ms, total: 16.7 s
Wall time: 4.19 s


,user_id,actual,als,als_tfidf,als_bm25
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5569374, 1033142, 979707, 1105488, 995242]","[1056509, 1041796, 844818, 1033142, 995242]","[940631, 1033142, 1056509, 965766, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 1133018, 910032, 1022003, 938700]","[938700, 1124729, 1022003, 951590, 1051069]","[1051069, 1022003, 9337581, 877358, 1055863]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1007195, 866211, 878996, 820165, 986912]","[1023720, 1007195, 1127179, 878996, 911878]","[1082185, 866211, 1051516, 986947, 919681]"
